In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def unique_words(s):
    unique = set(s.split(' ')) 
    return len(unique)

# def sentences(s):
#     sent = s.replace('|||',' ').split()
#     count = 0
#     for word in sent:
#         if len(word)>1:
#             if word[-1]=='.|!|?' :
#                 count+=1
#     return count

def emojis(post):
    # does not include emojis made purely from symbols, only :word:
    emoji_count = 0
    words = post.split()
    for e in words:
        if 'http' not in e:
            if e.count(':')==2:
                emoji_count+=1
    return emoji_count

def colons(post):
    # Includes colons used in emojis
    colon_count = 0
    words = post.split()
    for e in words:
        if 'http' not in e:
            colon_count+=e.count(':')
    return colon_count

def domains(post):
    # Includes colons used in emojis
    domains = []
    words = post.split()
    for s in words:
        if 'http' in s:
            try:
                s1 = s.split('/')[2]
    #             s1 = s.split('.com')[0].split('//')[-1]
    #             s2 = s1[s1.rfind('.')+1:]
                if s1 not in domains:
                    domains.append(s1)
            except:
                print(s)
    return domains

def grouper(df,group):
    return df.groupby(group).agg({
        'type':'count',
        'unique_words':['sum','mean','median'],
        'link_count':['sum','mean','median'],
        'word_count':['sum','mean','median'],
        'qm':['sum','mean','median'],
#         'sentences':['sum','mean','median']
    })

In [3]:
#Pull in csv
mbti_user = pd.read_csv('mbti_1.csv')
#.reset_index().rename(columns={'index':'id'})

In [4]:
mbti_user['unique_words'] = mbti_user['posts'].str.replace('|||', ' ').apply(unique_words)
mbti_user['link_count'] = mbti_user['posts'].apply(lambda s : s.count('http'))
mbti_user['word_count'] = mbti_user['posts'].apply(lambda s : s.count(' ')+1)
mbti_user['qm'] = mbti_user['posts'].apply(lambda s : s.count('?'))
mbti_user['em'] = mbti_user['posts'].apply(lambda s : s.count('!'))
# mbti_user['sentences'] = mbti_user['posts'].apply(sentences)

In [5]:
mbti_user['IE'] = mbti_user['type'].apply(lambda s : s[0])
mbti_user['NS'] = mbti_user['type'].apply(lambda s : s[1])
mbti_user['FT'] = mbti_user['type'].apply(lambda s : s[2])
mbti_user['JP'] = mbti_user['type'].apply(lambda s : s[3])

In [6]:
mbti_user.head()

,type,posts,unique_words,link_count,word_count,qm,em,IE,NS,FT,JP
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,77,24,578,18,3,I,N,F,J
1,ENTP,'I'm finding the lack of me in these posts ver...,77,10,1194,5,0,E,N,T,P
2,INTP,'Good one _____ https://www.youtube.com/wat...,72,5,869,12,4,I,N,T,P
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",74,2,1101,11,3,I,N,T,J
4,ENTJ,'You're fired.|||That's another silly misconce...,77,6,987,10,1,E,N,T,J


In [7]:
posts = mbti_user.apply(lambda x: pd.Series(x['posts'].split('|||')), axis=1).stack().reset_index(level=1, drop=True)
posts.name = 'post'
mbti_post = mbti_user[['type']].join(posts)
mbti_post['post'] = pd.Series(mbti_post['post'], dtype=object)
mbti_post = mbti_post.reset_index().rename(columns={'index':'id'})

In [8]:
mbti_post['unique_words'] = mbti_post['post'].apply(unique_words)
mbti_post['link_count'] = mbti_post['post'].apply(lambda s : s.count('http'))
mbti_post['word_count'] = mbti_post['post'].apply(lambda s : s.count(' ')+1)
mbti_post['qm'] = mbti_post['post'].apply(lambda s : s.count('?'))
mbti_post['em'] = mbti_post['post'].apply(lambda s : s.count('!'))
mbti_post['colons'] = mbti_post['post'].apply(colons)
mbti_post['emojis'] = mbti_post['post'].apply(emojis)
# mbti_post['sentences'] = mbti_post['post'].apply(sentences)

In [9]:
mbti_post.head()

,id,type,post,unique_words,link_count,word_count,qm,em,colons,emojis
0,0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw,1,1,1,1,0,0,0
1,0,INFJ,http://41.media.tumblr.com/tumblr_lfouy03PMA1q...,1,1,1,0,0,0,0
2,0,INFJ,enfp and intj moments https://www.youtube.com...,13,2,16,2,0,0,0
3,0,INFJ,What has been the most life-changing experienc...,10,0,10,1,0,0,0
4,0,INFJ,http://www.youtube.com/watch?v=vXZeYwwRDw8 h...,9,2,11,2,0,0,0


In [10]:
mbti_post['unique_domains'] = mbti_post['post'].apply(domains)
mbti_post['unique_domains_count'] = mbti_post['unique_domains'].apply(len)

http:youtube.com/watch?v=KAveNvDnL_A
https...
https
https/tls
http:
https


In [11]:
unique_domains = {}
for post in mbti_post['post']:
    if 'http' in post:
        for s in post.split():
            if 'http' in s:
                s1 = s.split('.com')[0].split('//')[-1]
                s2 = s1[s1.rfind('.')+1:]
                if s2 not in unique_domains:
                    unique_domains[s2] = 1
                else:
                    unique_domains[s2] += 1

In [12]:
pd.DataFrame(unique_domains.items(), columns=['domain', 'count']).sort_values('count', ascending=False)

,domain,count
0,youtube,16708
8,jpg,1246
21,imgur,962
1,tumblr,875
22,personalitycafe,687
...,...,...
1307,be/lG5aSZBAuPs,1
1308,be/jxRF8JqDaJ0,1
1309,be/wr3hgqfn4j0,1
1310,be/29Ydo8joLrE,1


In [13]:
mbti_post[mbti_post['post'].str.contains('youtu.be')]

,id,type,post,unique_words,link_count,word_count,qm,em,colons,emojis,unique_domains,unique_domains_count
212,4,ENTJ,http://youtu.be/PZbqAMEwtOE,1,1,1,0,0,0,0,[youtu.be],1
222,4,ENTJ,DEAD XD http://youtu.be/PaghIdSJKvQ,4,1,5,0,0,0,0,[youtu.be],1
693,14,INTP,'Basically this... https://youtu.be/1pH5c1JkhLU,4,1,4,0,0,0,0,[youtu.be],1
701,14,INTP,http://youtu.be/KtYkyB35zkk,1,1,1,0,0,0,0,[youtu.be],1
730,14,INTP,http://youtu.be/AJzds6fRfjM,1,1,1,0,0,0,0,[youtu.be],1
...,...,...,...,...,...,...,...,...,...,...,...,...
421392,8645,INFJ,There is only one thing to do - follow in the ...,29,1,30,1,0,1,0,[youtu.be],1
422321,8664,INTP,https://youtu.be/S34cVkL6zCE,1,1,1,0,0,0,0,[youtu.be],1
422333,8664,INTP,https://youtu.be/wTG4746_Fgc,1,1,1,0,0,0,0,[youtu.be],1
422624,8670,ISFP,https://youtu.be/aznd4gj5kaU,1,1,1,0,0,0,0,[youtu.be],1


In [14]:
s = 'http://youtu.be/PZbqAMEwtOE'
s.split('/')

['http:', '', 'youtu.be', 'PZbqAMEwtOE']